# Lista 3 - KNN e árvores de decisão

## Sumário

-   Questão 1: [A](#Questão-1-a.), [B](#Questão-1-b.)

<span style="position: absolute; top: 10px; right: 10px; background: green; padding: 0.5em; color: white; border-radius: 8px; font-weight: bold">Vaux Gomes</span>

## Implementações

### Importações


In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sklearn.preprocessing
from sklearn.base import \
    BaseEstimator, RegressorMixin
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

### Experimento


In [2]:
def cross_val(clf, X, y, folds=10, stats=False, progress=False):
    ''' Runs a full n-fold cross-validation on clf '''
    
    # Auxiliary
    stats = {
        'precision': [], 'acc': [], 
        'recall': [], 'fscore': []
    }
    p_size = X.shape[0]/folds # partition size (float)
    
    # Normalizer
    normalizer = MinMax()
    
    # Labels
    labels = np.unique(y_kc2)

    # Cross validation
    for n in range(folds):
        # Partition boundaries
        start, end = int(n * p_size), int((n + 1) * p_size)

        # Normalized Train
        X_train = normalizer.fit_transform(np.concatenate([X[:start], X[end:]]))
        y_train = np.concatenate([y[:start], y[end:]])

        # Normalized Validation
        X_valid = normalizer.transform(X[start:end])
        y_valid = y[start:end]
        
        # Training
        clf.fit(X_train, y_train)
        
        # Prediction
        y_hat = clf.predict(X_valid)

        # Confusion Matrix
        c_matrix = confusion_matrix(y_valid, y_hat, labels=labels)
        
        # True positive
        tp = c_matrix.diagonal()[0]
        # False positive + false Negative
        fpfn = np.rot90(c_matrix).diagonal().sum()
        
        # Precision
        precision = tp / c_matrix.sum(axis=0)[0]  # (TP + FB)
        stats['precision'].append(precision)

        # Accuracy
        acc = c_matrix.diagonal().sum()/c_matrix.sum()
        stats['acc'].append(acc)
        
        # Recall
        recall = tp / c_matrix.sum(axis=1)[0] # (TP + FN)
        stats['recall'].append(recall)
        
        # F1-score
        fscore = 2*tp / (2*tp + c_matrix.sum(axis=0)[0])
        stats['fscore'].append(fscore)

        # Progress printing for watching the magic happening
        if progress:
            print(end='-')   
        
    # Printing
    if stats:
        print('-'*(50 - (n if progress else 0)))
        print(f' {clf}')
        print('-'*50)
        print(f' Precision: {np.nanmean(stats["precision"]):.5} ± {np.nanstd(stats["precision"]):.5}')
        print(f' Accuracy.: {np.nanmean(stats["acc"]):.5} ± {np.nanstd(stats["acc"]):.5}')
        print(f' Recall...: {np.nanmean(stats["recall"]):.5} ± {np.nanstd(stats["recall"]):.5}')
        print(f' F1-score.: {np.nanmean(stats["fscore"]):.5} ± {np.nanstd(stats["fscore"]):.5}')
        
    # Return
    return c_matrix

### MinMax Normalizer


In [3]:
class MinMax():
    ''' MinMax Normalizer '''

    #
    def fit_transform(self, X):
        self.min = X.min(axis=0)
        self.max = X.max(axis=0)
        
        return (X - self.min)/(self.max - self.min)
    
    #
    def transform(self, X):
        return (X - self.min)/(self.max - self.min)
    
    #
    def restore(self, X):
        return X*(self.max - self.min) + self.min

### KNN


In [4]:
class KNN(BaseEstimator, RegressorMixin):
    ''' K-nearest neighbors '''
    
    EUCLIDIAN = 'euclidian'
    MAHALANOBIS = 'mahalanobis'

    #
    def __init__(self, k=1, f_distance='euclidian', solve_ties=False):
        ''' solve_ties: allow decreasing the value of k when ties occur '''


        if f_distance.lower() not in [self.EUCLIDIAN, self.MAHALANOBIS]:
            raise Exception(f'{f_distance} is not a valid value for f_distance')
        
        self.k = k
        self.solve_ties = solve_ties
        self.f_distance = f_distance.lower()
        
    # 
    def fit(self, X, y):
        self.X = X
        self.y = y

        #
        n_classes = np.unique(y).size

        # Safety
        if n_classes <= 1:
            raise Exception('Warning: your dataset must have more than 1 class')

        # Friendly warning
        if n_classes % 2 == 0 and self.k % 2 == 0:
            warnings.warn(f'Warning: your dataset has {n_classes} classes and you k is equals to {self.k}. It is recommended to choose and odd value for k to avoid ties')
         
    #
    def predict(self, X):        
        # Output
        Y_hat = []

        # Distance function
        f_distance = KNN.__euclidian_distance

        if self.f_distance != self.EUCLIDIAN:
            f_distance = KNN.__mahalanobis_distance

        #
        for x in X:
            distances = f_distance(self.X, x)
            distances_sidx = np.argsort(distances)    # Ascending
            
            #
            j = 0
            l = len(Y_hat)

            # Untying if necessary
            while j < self.k and l == len(Y_hat):
                classes, counts = np.unique(self.y[distances_sidx[:self.k-j]], 
                    return_counts=True)

                # No ties
                if counts.size == 1:
                    Y_hat.append(classes[0])
                # Possible tie
                else:
                    # Careless approach
                    if not self.solve_ties:
                        Y_hat.append(classes[0])
                    # Solve tie
                    else:
                        counts_sidx = np.argsort(-counts)

                        # Check tie                    
                        if counts[counts_sidx[0]] == counts[counts_sidx[1]]:
                            j += 1
                        else:
                            Y_hat.append(classes[counts_sidx[0]])
        
        return Y_hat

    #
    def __euclidian_distance(X, x):
        return np.linalg.norm(X  - x, axis=1)
    
    def __mahalanobis_distance(X, x):
        return np.sqrt((x - np.mean(X, axis=0))**2 / np.var(x, axis=0)) # It is a bit expensive

---

## Questão 1

Considere o conjunto de dados disponível em `kc2.csv`, organizado em 22 colunas, sendo as 21 primeiras colunas os atributos e a última coluna a saída. Os 21 atributos são referentes à caracterização de códigos-fontes para processamento de dados na NASA. A saída é a indicação de ausência (0) ou existência (1) de
defeitos (os dados foram balanceados via subamostragem).


In [5]:
# Importação dos conjuntos de dados
kc2 = np.genfromtxt('kc2.csv', delimiter=',')
np.random.shuffle(kc2)

print(f'KC2: {kc2.shape}')
print('ATENÇÃO OS DADOS FORAM EMBARALHADOS')

KC2: (214, 22)
ATENÇÃO OS DADOS FORAM EMBARALHADOS


> Dados embaralhados devido a distribuição de classes, mas funciona normalmente sem este embaralhamento

### Questão 1 a.

Considerando uma validação cruzada em 10 folds, avalie modelos de classificação binária nos dados em questão. Para tanto, use as abordagens abaixo:

-   KNN
    -   $k = 1$ e distância Euclidiana
    -   $k = 1$ e distância Mahalanobis distance
    -   $k = 5$ e distância Euclidiana
    -   $k = 5$ e distância Mahalanobis distance
-   Árvore de decisão (você pode usar uma implementação já existente
    com índices de impureza de gini e entropia).

$$\displaystyle\mbox{Mahalanobis} = \sqrt{\frac{(\mbox{x} - \bar{x})^2}{\sigma^2}}$$

### Questão 1 b.

Para cada modelo criado, reporte valor médio e desvio padrão das métricas de **acurácia**, **revocação**, **precisão** e **F1-score**

[Início ↑](#Sumário)


In [6]:
X_kc2 = kc2[:, :-1]
y_kc2 = kc2[:, -1:]

#### KNN (Euclidian)


In [7]:
clf = KNN(k=1)
_ = cross_val(clf, X_kc2, y_kc2, stats=True, progress=True)

---------------------------------------------------
 KNN()
--------------------------------------------------
 Precision: 0.75409 ± 0.13996
 Accuracy.: 0.71385 ± 0.098159
 Recall...: 0.68665 ± 0.12732
 F1-score.: 0.59535 ± 0.053811


In [8]:
clf = KNN(k=5)
_ = cross_val(clf, X_kc2, y_kc2, stats=True, progress=True)

---------------------------------------------------
 KNN(k=5)
--------------------------------------------------
 Precision: 0.59893 ± 0.11264
 Accuracy.: 0.65022 ± 0.091996
 Recall...: 0.94246 ± 0.063709
 F1-score.: 0.5402 ± 0.047015


#### KNN (Mahalanobis)

In [9]:
clf = KNN(k=1, f_distance=KNN.MAHALANOBIS)
_ = cross_val(clf, X_kc2, y_kc2, stats=True, progress=True)

---------------------------------------------------
 KNN(f_distance='mahalanobis')
--------------------------------------------------
 Precision: 0.50771 ± 0.12259
 Accuracy.: 0.5145 ± 0.099807
 Recall...: 0.81406 ± 0.17987
 F1-score.: 0.49473 ± 0.075923


In [10]:
clf = KNN(k=5, f_distance=KNN.MAHALANOBIS)
_ = cross_val(clf, X_kc2, y_kc2, stats=True, progress=True)

---------------------------------------------------
 KNN(f_distance='mahalanobis', k=5)
--------------------------------------------------
 Precision: 0.50173 ± 0.11204
 Accuracy.: 0.50173 ± 0.11204
 Recall...: 1.0 ± 0.0
 F1-score.: 0.49463 ± 0.056145


#### Árvore de decisão


In [11]:
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
from sklearn.tree import DecisionTreeClassifier as TreeClf

In [12]:
clf = TreeClf()
_ = cross_val(clf, X_kc2, y_kc2, stats=True, progress=True)

---------------------------------------------------
 DecisionTreeClassifier()
--------------------------------------------------
 Precision: 0.68032 ± 0.1269
 Accuracy.: 0.69675 ± 0.092399
 Recall...: 0.75361 ± 0.13272
 F1-score.: 0.57128 ± 0.047602


In [14]:
clf = TreeClf(criterion="entropy")
cm = cross_val(clf, X_kc2, y_kc2, stats=True, progress=True)

---------------------------------------------------
 DecisionTreeClassifier(criterion='entropy')
--------------------------------------------------
 Precision: 0.68601 ± 0.12058
 Accuracy.: 0.69156 ± 0.081761
 Recall...: 0.73793 ± 0.14091
 F1-score.: 0.57371 ± 0.046425
